In [ ]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
import logging

#choices:  CPU, GPU0...GPU7
# Try GPU first, fallback to CPU if not available
try:
    device.set(device=DeviceId.GPU0)
    device_info = device.get_device_info()
    print(f"Device setup: {device_info}")
    if device_info['is_gpu']:
        print(f"✅ Using GPU: {device_info.get('cuda_device_name', 'Unknown GPU')}")
    else:
        print("⚠️ Using CPU (GPU not available)")
except Exception as e:
    print(f"Device setup error: {e}")
    device.set(device=DeviceId.CPU)
    print("🔄 Fallback to CPU")

In [ ]:
from deoldify.visualize import *
from deoldify.enhanced_video_colorizer import EnhancedVideoColorizer, MultiModelVideoColorizer
plt.style.use('dark_background')
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

In [ ]:
# Initialize enhanced video colorizers with multiple models
print("🔄 Loading video colorization models...")

# Get base colorizers
video_vis = get_video_colorizer(render_factor=21)
stable_vis = get_stable_video_colorizer(render_factor=21) 
artistic_vis = get_artistic_video_colorizer(render_factor=35)

# Create enhanced colorizers with temporal consistency
video_enhanced = EnhancedVideoColorizer(
    video_vis.vis,
    enable_temporal_consistency=True,
    enable_edge_enhancement=True,
    enable_color_stabilization=True
)

stable_enhanced = EnhancedVideoColorizer(
    stable_vis.vis,
    enable_temporal_consistency=True,
    enable_edge_enhancement=True,
    enable_color_stabilization=True
)

artistic_enhanced = EnhancedVideoColorizer(
    artistic_vis.vis,
    enable_temporal_consistency=True,
    enable_edge_enhancement=True,
    enable_color_stabilization=True
)

# Create multi-model colorizer
multi_colorizer = MultiModelVideoColorizer()
multi_colorizer.add_colorizer("video", video_enhanced)
multi_colorizer.add_colorizer("stable", stable_enhanced)
multi_colorizer.add_colorizer("artistic", artistic_enhanced)

print("✅ Enhanced video colorizers loaded successfully!")
print(f"Available methods: {multi_colorizer.get_available_methods()}")

# Enhanced Video Colorization Instructions

This notebook now uses **Enhanced DeOldify** with several improvements:

## 🚀 New Features:
- **Temporal Consistency**: Reduces flickering between frames using optical flow
- **Edge Enhancement**: Preserves edges and reduces color bleeding
- **Color Stabilization**: Maintains consistent color palettes throughout the video
- **Multiple Models**: Choose between Video, Stable, and Artistic colorization methods
- **Grayscale Conversion**: Frames are properly converted to grayscale before colorization
- **GPU Detection**: Automatically detects and uses GPU when available

## 📁 Parameters:

### source_url
Type in a url hosting a video from YouTube, Imgur, Twitter, Reddit, Vimeo, etc. Many sources work! GIFs also work. Full list here: https://ytdl-org.github.io/youtube-dl/supportedsites.html 

**NOTE**: If you want to use your own video, you can set source_url to None and just upload the file to video/source/ in Jupyter. Just make sure that the file_name parameter matches the file you uploaded.

### file_name
Name this whatever sensible file name you want (minus extension)! It should actually exist in video/source if source_url=None

### colorization_method
- **"video"**: Specialized for video sequences (default)
- **"stable"**: More realistic, stable colors
- **"artistic"**: More vibrant and creative colors

### render_factor
The default value of 21 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the video is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality film in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality videos and inconsistencies (flashy render) will generally be reduced, but the colors may get slightly washed out.

### Enhancement Options
- **temporal_consistency**: Reduces flickering (recommended: True)
- **edge_enhancement**: Preserves edges (recommended: True) 
- **color_stabilization**: Maintains color consistency (recommended: True)
- **post_processing_preset**: "balanced", "vivid", "soft", "sharp", "cinematic"
- **frame_skip**: Process every Nth frame (1=all frames, 2=every other frame, etc.)

## 💡 Pro Tips:
1. **GPU Usage**: The notebook now automatically detects and uses GPU when available
2. **Better Quality**: Enhanced processing reduces artifacts and improves consistency
3. **Frame Preview**: Use the bottom cell to preview how different render_factors perform
4. **Method Selection**: Try different colorization methods for different types of content

## 🔧 Troubleshooting:
- **GPU Issues**: Check the first cell output for GPU detection status
- **Memory Issues**: Reduce render_factor or enable frame_skip
- **Quality Issues**: Try different colorization methods or post-processing presets

## 🎨 Enhanced Colorization

In [ ]:
# Enhanced colorization parameters
render_factor = 21  # NOTE: Max is 44 with 11GB video cards. 21 is a good default
colorization_method = "video"  # Options: "video", "stable", "artistic"
source_url = 'https://twitter.com/silentmoviegifs/status/1116751583386034176'
file_name = 'DogShy1926'
file_name_ext = file_name + '.mp4'

# Enhancement options
temporal_consistency = True      # Reduces flickering between frames
edge_enhancement = True          # Preserves edges and reduces color bleeding
color_stabilization = True       # Maintains consistent colors
post_processing_preset = "balanced"  # Options: "balanced", "vivid", "soft", "sharp", "cinematic"
frame_skip = 1                   # Process every N frames (1=all frames, 2=every other frame)

print(f"🎬 Starting enhanced video colorization...")
print(f"Method: {colorization_method}")
print(f"Render factor: {render_factor}")
print(f"Temporal consistency: {temporal_consistency}")
print(f"Edge enhancement: {edge_enhancement}")
print(f"Color stabilization: {color_stabilization}")
print(f"Post-processing: {post_processing_preset}")
print(f"Frame skip: {frame_skip}")

if source_url is not None:
    # Download and colorize from URL
    source_path = multi_colorizer.colorizers[colorization_method].source_folder / file_name_ext
    multi_colorizer.colorizers[colorization_method]._download_video_from_url(source_url, source_path)
    result_path = multi_colorizer.colorize_video(
        file_name_ext, 
        method=colorization_method,
        render_factor=render_factor,
        temporal_consistency=temporal_consistency,
        edge_enhancement=edge_enhancement,
        color_stabilization=color_stabilization,
        post_processing_preset=post_processing_preset,
        frame_skip=frame_skip
    )
else:
    # Colorize from local file
    result_path = multi_colorizer.colorize_video(
        file_name_ext,
        method=colorization_method, 
        render_factor=render_factor,
        temporal_consistency=temporal_consistency,
        edge_enhancement=edge_enhancement,
        color_stabilization=color_stabilization,
        post_processing_preset=post_processing_preset,
        frame_skip=frame_skip
    )

print(f"✅ Enhanced colorization complete!")
print(f"📁 Output saved to: {result_path}")
show_video_in_notebook(result_path)

## 🔍 Compare Different Methods

In [ ]:
# Compare different colorization methods on the same video
methods_to_compare = ["video", "stable", "artistic"]
comparison_results = {}

for method in methods_to_compare:
    print(f"\n🎨 Processing with {method} method...")
    try:
        result = multi_colorizer.colorize_video(
            file_name_ext,
            method=method,
            render_factor=21,
            temporal_consistency=True,
            edge_enhancement=True,
            color_stabilization=True,
            frame_skip=2  # Skip frames for faster comparison
        )
        comparison_results[method] = result
        print(f"✅ {method} method complete: {result}")
    except Exception as e:
        print(f"❌ {method} method failed: {e}")

print(f"\n🏁 Comparison complete! Results:")
for method, path in comparison_results.items():
    print(f"- {method}: {path}")

## 🔬 Preview Render Factor Performance

In [ ]:
# Preview how different render_factor values perform on a single frame
print("🔍 Testing different render factors on sample frame...")

# Use the first frame from the extracted frames
bwframes_folder = f'video/bwframes/{file_name}'
sample_frame = f'{bwframes_folder}/00001.jpg'

# Test different render factors
test_factors = range(15, 35, 3)  # Test factors from 15 to 32, step by 3

for i in test_factors:
    print(f"Testing render factor: {i}")
    try:
        multi_colorizer.colorizers[colorization_method].vis.plot_transformed_image(
            sample_frame, 
            render_factor=i, 
            display_render_factor=True, 
            figsize=(8,8)
        )
    except Exception as e:
        print(f"Error with render factor {i}: {e}")

print("✅ Render factor testing complete!")

## 📊 Device and Performance Information

In [ ]:
# Display current device and performance information
device_info = device.get_device_info()
print("💻 Device Information:")
for key, value in device_info.items():
    print(f"  {key}: {value}")

print(f"\n🎯 Available colorization methods: {multi_colorizer.get_available_methods()}")
print(f"📁 Video processing folders:")
print(f"  Source: video/source/")
print(f"  Grayscale frames: video/bwframes/")
print(f"  Colorized frames: video/colorframes/")
print(f"  Results: video/result/")